In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from debug_utils import get_omodel, get_peft_ckpt, generate_outputs, adjust_input_apf, adjust_input_stack
from datasets import load_dataset

/home/prasann/miniconda3/envs/rlenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-08-17 17:51:20,477] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Start by checking stuff out for the webgpt model

In [4]:
with open('../basepath.txt', 'r') as file:
    bpath = file.read().strip()

with open('../apfpath.txt', 'r') as file:
    apfpath = file.read().strip()

In [ ]:
# for webgpt 
basename = apfpath+"apfarm_models/sft10k/"
origmodel, toker = get_omodel(basename)

In [5]:
# for stack stuff
basename = bpath+"stack-llama/models/sft/"
origmodel, toker = get_omodel(basename)

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.05s/it]


In [6]:
generation_kwargs = {
    "min_new_tokens": -1,
    "max_new_tokens":512,
    #"top_k": 0.0,
    "top_p": 1,
    #"temperature": 0.9,
    "do_sample": True,
    #"repetition_penalty": 1.2,
    #"pad_token_id": tokenizer.pad_token_id,
    #"eos_token_id": tokenizer.eos_token_id,
}

In [ ]:
# for webgpt
inps = load_dataset("openai/webgpt_comparisons", split="test")

In [7]:
inps = load_dataset("lvwerra/stack-exchange-paired", data_dir="data/evaluation", split="train")

In [53]:
ind = 20
qtemp = adjust_input_apf(inps[ind]['question']['full_text'])
toks, outs = generate_outputs([qtemp], toker, origmodel, generation_kwargs)
print(len(toks[0]))
print(outs[0])

<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
How many volunteer hours were estimated to have supported the 2012 Summer Olympics?

### Response:Approximately 70,000 volunteer hours are estimated to have been needed to support the 2012 Summer Olympics. This is equivalent to about 280 volunteers working full-time for one year. 70,000 volunteers were recruited for the Games, and they were divided into four categories: Gamesmaker, Venue Volunteer, Event Volunteer and Media Volunteer. The Gamesmakers, Venue Volunteers, and Event Volunteers were responsible for helping with various activities during the Games such as providing information to spectators, controlling access to venues, and manning various events. Media Volunteers were responsible for assisting with broadcasting, media operations, and photography. Volunteers were required to attend training sessions and adhere to a code of conduct, and they were gi

In [ ]:
#SFT Starts out reasonably short

In [60]:
adaptpath = bpath+"/webgpt-llama/checkpoints/wgptapsft/step_125"
amod = get_peft_ckpt(origmodel, adaptpath)

In [62]:
ind = 40
qtemp = adjust_input_apf(inps[ind]['question']['full_text'])
toks, outs = generate_outputs([qtemp], toker, amod, generation_kwargs)
print(len(toks[0]))
print(outs[0])

330
<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Name the 1980's hit sung by Tina Turner and Rod Stewart?

### Response:The 1980's hit sung by Tina Turner and Rod Stewart is "Simply the Best". It was written and composed by Mike Chapman and Holly Knight, and released as a single from Tina Turner's 1984 album, Break Every Rule. The song was also included as a track on Rod Stewart's album Outshined in the same year. The single reached number one on the US Billboard Hot 100 chart and on the UK Singles Chart, respectively, becoming one of Turner's signature songs. It was also nominated for the Grammy Award for Best Female Rock Vocal Performance in 1985. The song is considered to be one of the greatest songs of the 80's and is often seen as Turner's signature song. Turner also performs the song at her concerts, and it is one of the songs Stewart most frequently performs in his shows. Despite its success, the 

In [8]:
# Testing stuff for Stack Model (OOF the KL was too high to be a reproduction...)

In [15]:
lens = []
for ind in range(0, 20):
    qtemp = adjust_input_stack(inps[ind]['question'])
    toks, outs = generate_outputs([qtemp], toker, origmodel, generation_kwargs)
    print(len(toks[0]))
    print(outs[0])
    lens.append(len(toker(outs[0].split("Answer:")[1]).input_ids))

511
<s> Question: I have to go into a table to retrieve a parameter, then go back into the same table to retrieve data based on the parameter.

```
<cfquery name = "selnm" datasource = "Moxart">
 select SelName from AuxXref 
 where Fieldname = <cfqueryparam value = "#orig#">
</cfquery>

<cfset selname = selnm.SelName>

<cfquery name = "fld" datasource = "Moxart">
select Fieldname, DBname from AuxXref 
where SelName = <cfqueryparam value = "#selname#">
</cfquery>

```

Can this be done in a single query?

Answer: 1. <cfloop> and <cfif/> conditional looping
2. [Conditional Aggregate](https://help.sap.com/saphelp_bw405web/helpdata/en/2/fa/d7fbf99e39e49c59fc73558aa3bcc9/content.htm?frameset=/en/2/fa/d7fbf99e39e49c59fc73558aa3bcc9/frameset.htm&current_toc=/en/2/fa/d7fbf99e39e49c59fc73558aa3bcc9/toc.htm&bookmark=toc1.htm)
3. [User-defined functions](https://help.sap.com/saphelp_bw405web/helpdata/en/3/2a/eba1d697085c5c0b2b1ae4b5bccdd4/content.htm?frameset=/en/3/2a/eba1d697085c5c0b2b1ae4b5bccd


KeyboardInterrupt



In [22]:
adaptpath = bpath+"/stack-llama/checkpoints/lenhacksanity/step_125"
amod = get_peft_ckpt(origmodel, adaptpath)

In [23]:
lens = []
for ind in range(0, 20):
    qtemp = adjust_input_stack(inps[ind]['question'])
    toks, outs = generate_outputs([qtemp], toker, origmodel, generation_kwargs)
    print(len(toks[0]))
    print(outs[0])
    lens.append(len(toker(outs[0].split("Answer:")[1]).input_ids))

451
<s> Question: I have installed the Java 3D API on PC via the exe installer, which simply created a new directory with `j3dcore.jar`, `vecmath.jar`, `j3dutils.jar` in a lib sub-directory and `j3dcore-ogl.dll` in a bin sub-directory.

Netbeans had no issues and my code compiled and executed smoothly, however once I built my project and tried to run it from the command prompt I got an `UnsatisfiedLinkError` saying that `no j3dcore-ogl in java.library.path`. 

Google came to the rescue and gave me 3 viable solutions:

* by copying the dll file into my JRE's bin directory
* by adding the path of the dll file to the library path (`java -Djava.library.path=dllpath`)
* load the dll in the program with `System.load()` (I couldn't get this one to work, actually)

My question is: Is there an elegant solution to this problem, that I missed? 

It seems tedious that for each different PC someone would like to use this program on, he'd have to either copy the dll or add it to the library path bef


KeyboardInterrupt



In [24]:
sum(lens)/len(lens)

355.75

In [25]:
lens

[150, 514, 199, 514, 380, 379, 354, 272, 180, 513, 300, 514]